<a href="https://www.kaggle.com/code/mohammedhannnane/real-estate-analysis-using-sql?scriptVersionId=158311384" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Real Estate Analysis using SQL 

This project focuses on amazing USA real estate data using SQL and creating an interactive Tablleau ddashboard. Through SQL, we'll examine property prices, trends, and geographical distribution, aiming to uncover key insights. The resulting Tableau dashboard will offer a user-friendly interface, providing a comprehensive visual representation of the intricate dynamics within the real estate market. 

# Visualization using Tableau

https://public.tableau.com/app/profile/mohammed.hannane/viz/USARealEstateDataAnalysis_17048008770820/Dashboard



# Calculate the price difference between consecutive sales within each state, excluding duplicated rows
with engine.begin() as conn:
    query = text("""
    SELECT state, prev_sold_date, price,
           price - LAG(price) OVER (PARTITION BY state ORDER BY prev_sold_date) AS price_difference
    FROM (
        SELECT state, prev_sold_date, price,
               ROW_NUMBER() OVER (PARTITION BY state, price, prev_sold_date ORDER BY price) AS row_num
        FROM realtor
        WHERE prev_sold_date IS NOT NULL
        ) sub
    WHERE row_num = 1
    ORDER BY state, prev_sold_date;
    """)
    df13 = pd.read_sql_query(query, conn)
    
df13



In [1]:



# Load packages
import pandas as pd 
import sqlite3
from sqlalchemy import create_engine, text

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))




/kaggle/input/usa-real-estate-dataset/realtor-data.zip.csv


In [2]:
# Load data
df = pd.read_csv('/kaggle/input/usa-real-estate-dataset/realtor-data.zip.csv')

In [3]:
# Create a sqlite engine 
engine = create_engine('sqlite://', echo=False)

# Explore the dataframe as a table to the sqlite engine
df.to_sql("realtor", con=engine, index=False)

1401066

In [4]:
# How many properties are listed for sale by state?
with engine.begin() as conn:
    query = text("""
    SELECT state,
    COUNT(*) AS properties_count
    FROM realtor
    WHERE status = 'for_sale' 
    GROUP BY state
    ORDER BY properties_count DESC
    """)
    df1 = pd.read_sql_query(query, conn)
    
df1

,state,properties_count
0,New York,652914
1,New Jersey,255663
2,Massachusetts,176410
3,Connecticut,98804
4,New Hampshire,51394
5,Vermont,48230
6,Maine,36650
7,Rhode Island,29610
8,Puerto Rico,24679
9,Pennsylvania,19978


In [5]:
# Average number of bedrooms and bathrooms for properties that are ready to sale 
with engine.begin() as conn:
    query = text("""
    SELECT AVG(bed) AS avg_bedrooms,
AVG(bath) AS avg_bathrooms
FROM realtor
WHERE status = 'for_sale';
""")
    df2 = pd.read_sql_query(query, conn)
    
df2

,avg_bedrooms,avg_bathrooms
0,3.393161,2.487426


In [6]:
# Average, min and max prices by state
with engine.begin() as conn:
    query = text("""
    SELECT state,
    ROUND(AVG(price)) AS avg_price,
    MIN(price) AS min_price,
    MAX(price) AS max_price
    FROM realtor
    GROUP BY state
    ORDER BY state;
    """)
    df3 = pd.read_sql_query(query, conn)
    
df3

,state,avg_price,min_price,max_price
0,Connecticut,598812.0,6000.0,100000000.0
1,Delaware,345207.0,14900.0,3950000.0
2,Georgia,492704.0,145000.0,531625.0
3,Louisiana,875000.0,875000.0,875000.0
4,Maine,420010.0,7000.0,9000000.0
5,Massachusetts,970693.0,1.0,30000000.0
6,New Hampshire,480768.0,385.0,19500000.0
7,New Jersey,626098.0,0.0,25000000.0
8,New York,1005258.0,0.0,875000000.0
9,Pennsylvania,406505.0,4000.0,34000000.0


In [7]:
# Median price by state
with engine.begin() as conn:
    query = text("""
    SELECT state, AVG(price) AS median_price
    FROM (
    SELECT state, price,
    ROW_NUMBER() OVER(PARTITION BY state ORDER BY price) AS row_num,
    COUNT(*) OVER (PARTITION BY state) AS total_count
FROM realtor
) AS sub
WHERE row_num IN((total_count + 1)/2,(total_count + 2)/2)
GROUP BY state;
    """)
    df4 = pd.read_sql_query(query, conn)

df4

,state,median_price
0,Connecticut,300000.0
1,Delaware,287900.0
2,Georgia,490225.0
3,Louisiana,875000.0
4,Maine,249900.0
5,Massachusetts,599900.0
6,New Hampshire,349900.0
7,New Jersey,450000.0
8,New York,499000.0
9,Pennsylvania,280000.0


In [8]:
# Average price for square feet for different states
with engine.begin() as conn:
    query = text("""
    SELECT state,
    ROUND(AVG(price / house_size)) AS avg_price_per_sqft
    FROM realtor 
    WHERE house_size IS NOT NULL
    GROUP BY state
    ORDER BY state;
    """)
    df5 = pd.read_sql_query(query, conn)
    
df5

,state,avg_price_per_sqft
0,Connecticut,239.0
1,Delaware,173.0
2,Georgia,150.0
3,Maine,262.0
4,Massachusetts,500.0
5,New Hampshire,262.0
6,New Jersey,292.0
7,New York,521.0
8,Pennsylvania,222.0
9,Puerto Rico,187.0


In [9]:
# Top 10 most expensive cities by average price 
with engine.begin() as conn:
    query = text("""
    SELECT city, ROUND(AVG(price), 0) AS avg_price
    FROM realtor
    GROUP BY city 
    ORDER BY avg_price DESC
    LIMIT 10;
    """)
    df6 = pd.read_sql_query(query,conn)

df6

,city,avg_price
0,Sagaponack,14134730.0
1,Waterfront,12000000.0
2,Kattskill Bay,10000000.0
3,Rochdale Village,9800000.0
4,Middletown Township,9200000.0
5,Bridgehampton,9004120.0
6,Waccabuc,8864815.0
7,Woods Hole,8583000.0
8,Watermill,7926518.0
9,Wainscott,7610483.0


In [10]:
# List of properties that is more expensive than average price in this city 
with engine.begin() as conn:
    query = text("""
    WITH  city_avg_price AS (
    SELECT city, ROUND(AVG(price)) AS avg_price
    FROM realtor
    GROUP BY city
    )
    SELECT rd.*,
                cap.avg_price AS city_average_price
    FROM realtor rd
    JOIN city_avg_price cap ON rd.city = cap.city
    WHERE rd.price > cap.avg_price;
    """)
    df7 = pd.read_sql_query(query,conn)

df7

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,city_average_price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,None,105000.0,92879.0
1,for_sale,5.0,3.0,7.46,Las Marias,Puerto Rico,670.0,5403.0,None,300000.0,173559.0
2,for_sale,3.0,2.0,0.08,Juana Diaz,Puerto Rico,795.0,1045.0,None,150000.0,115241.0
3,for_sale,5.0,5.0,0.74,Ponce,Puerto Rico,731.0,2677.0,None,649000.0,175336.0
4,for_sale,4.0,4.0,0.22,Mayaguez,Puerto Rico,680.0,3450.0,None,235000.0,176220.0
...,...,...,...,...,...,...,...,...,...,...,...
497301,for_sale,NaN,NaN,32.20,Forestville,New York,14062.0,NaN,None,164900.0,125300.0
497302,for_sale,3.0,4.0,4.40,North Collins,New York,14111.0,1902.0,2011-05-24,299999.0,223030.0
497303,for_sale,4.0,3.0,12.20,Angola,New York,14006.0,2500.0,2006-10-06,459900.0,171436.0
497304,for_sale,4.0,2.0,0.36,Silver Creek,New York,14136.0,2026.0,2000-09-01,187900.0,145784.0


In [11]:
# Find mean, min and max houses sizes by state
with engine.begin() as conn:
    query = text("""
    SELECT state, 
    ROUND(AVG(house_size),0) AS avg_size, 
    MIN(house_size) AS min_size,
    MAX(house_size) AS max_size
    FROM realtor
    WHERE house_size IS NOT NULL
    GROUP BY state
    ORDER BY state;
    """)
    df8 = pd.read_sql_query(query, conn)
    
df8

,state,avg_size,min_size,max_size
0,Connecticut,2196.0,100.0,99999.0
1,Delaware,2015.0,440.0,11725.0
2,Georgia,3389.0,3116.0,3661.0
3,Maine,2141.0,128.0,35388.0
4,Massachusetts,2285.0,104.0,99999.0
5,New Hampshire,2257.0,273.0,21500.0
6,New Jersey,2384.0,165.0,400149.0
7,New York,2096.0,4.0,352836.0
8,Pennsylvania,1874.0,100.0,74340.0
9,Puerto Rico,1942.0,181.0,1450112.0


In [12]:

# Properties with latest sold date for each city
with engine.begin() as conn:
    query = text("""
    WITH earliest_sold_properties AS (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY city ORDER BY prev_sold_date DESC) AS property_rank
        FROM realtor
        WHERE prev_sold_date IS NOT NULL
    )
    SELECT *
    FROM earliest_sold_properties
    WHERE property_rank = 1
    ORDER BY city, prev_sold_date DESC;
    """)
    df9 = pd.read_sql_query(query, conn)
    
df9



,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,property_rank
0,for_sale,4.0,2.0,0.18,Aberdeen,New Jersey,7747.0,NaN,2022-01-04,525000.0,1
1,for_sale,3.0,2.0,0.15,Abington,Pennsylvania,19001.0,1485.0,2017-10-23,329900.0,1
2,for_sale,3.0,2.0,0.25,Absecon,New Jersey,8201.0,NaN,2022-01-27,349000.0,1
3,for_sale,NaN,NaN,4.28,Absecon Highlands,New Jersey,8205.0,NaN,1985-12-24,89000.0,1
4,for_sale,NaN,NaN,1.60,Accord,New York,12404.0,NaN,2022-04-25,65000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2804,for_sale,3.0,2.0,0.33,Yonkers,New York,10710.0,1638.0,2021-11-12,590000.0,1
2805,for_sale,3.0,2.0,0.68,York,Maine,3909.0,2688.0,2021-03-19,439000.0,1
2806,for_sale,NaN,NaN,0.97,Yorktown Heights,New York,10598.0,NaN,2021-09-24,149000.0,1
2807,for_sale,NaN,NaN,0.13,Yorkville,New York,13495.0,NaN,2005-09-07,29900.0,1


In [13]:
# Create categories for bedrooms and count amount of properties in each category
with engine.begin() as conn:
    query = text("""
    SELECT bedroom_category, COUNT(*) AS property_count
    FROM (
        SELECT *,
               CASE
                   WHEN bed BETWEEN 1 AND 2 THEN 'small'
                   WHEN bed BETWEEN 2 AND 3 THEN 'medium'
                   ELSE 'large'
               END AS bedroom_category
        FROM realtor
    ) subquery
    GROUP BY bedroom_category;
    """)
    df10 = pd.read_sql_query(query, conn)
    
df10

,bedroom_category,property_count
0,large,665040
1,medium,379816
2,small,356210


In [14]:
# Percentage of expensive properties for each number of bathrooms 
with engine.begin() as conn:
    query = text("""
    SELECT bath,
        COUNT(*) AS property_count,
        AVG(CASE WHEN price > 500000 THEN 1.0 ELSE 0.0 END) * 100 AS percentage_expensive_properties
        FROM realtor
        WHERE bath IS NOT NULL 
        GROUP BY bath
        ORDER BY bath;
    """)
    df11 = pd.read_sql_query(query, conn)

df11

,bath,property_count,percentage_expensive_properties
0,1.0,300383,24.764717
1,2.0,429584,37.537245
2,3.0,285176,61.968749
3,4.0,104038,81.734559
4,5.0,41309,91.154470
5,6.0,20647,91.766358
6,7.0,9458,95.485303
7,8.0,6468,96.954236
8,9.0,3365,97.800892
9,10.0,2288,96.634615


In [15]:
# What is the cumulative sum of the house sizes for properties in each combination of city and state , ordered by the cumulative sum in descending order?
with engine.begin() as conn:
    query = text("""
    SELECT DISTINCT city, state,
            SUM(house_size) OVER (PARTITION BY city, state) AS cumulative_sum
            FROM realtor
            WHERE house_size IS NOT NULL
            ORDER BY cumulative_sum DESC;
    """)
    df12 = pd.read_sql_query(query, conn)

df12

,city,state,cumulative_sum
0,New York City,New York,74506668.0
1,Boston,Massachusetts,56254172.0
2,New York,New York,52525732.0
3,Brooklyn,New York,45599848.0
4,Bronx,New York,31481255.0
...,...,...,...
3959,Lexington Township,Maine,600.0
3960,Maxfield,Maine,600.0
3961,Lawton,Pennsylvania,532.0
3962,Wesley,Maine,448.0


In [16]:

# Calculate the price difference between consecutive sales within each state, excluding duplicated rows
with engine.begin() as conn:
    query = text("""
    SELECT state, prev_sold_date, price,
           price - LAG(price) OVER (PARTITION BY state ORDER BY prev_sold_date) AS price_difference
    FROM (
        SELECT state, prev_sold_date, price,
               ROW_NUMBER() OVER (PARTITION BY state, price, prev_sold_date ORDER BY price) AS row_num
        FROM realtor
        WHERE prev_sold_date IS NOT NULL
        ) sub
    WHERE row_num = 1
    ORDER BY state, prev_sold_date;
    """)
    df13 = pd.read_sql_query(query, conn)

df13

,state,prev_sold_date,price,price_difference
0,Connecticut,1961-04-04,2100000.0,NaN
1,Connecticut,1963-02-23,339000.0,-1761000.0
2,Connecticut,1965-03-12,309900.0,-29100.0
3,Connecticut,1965-11-05,329000.0,19100.0
4,Connecticut,1966-03-09,299900.0,-29100.0
...,...,...,...,...
78669,Virgin Islands,2010-01-26,129000.0,NaN
78670,Virgin Islands,2013-10-11,950000.0,821000.0
78671,Virgin Islands,2016-04-28,79500.0,-870500.0
78672,Virgin Islands,2016-11-16,176500.0,97000.0
